In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install hdf5plugin

In [ ]:
!pip install open3D

In [ ]:
import numpy as np
import os
import sys
import math
import h5py
import hdf5plugin
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import matplotlib.pyplot as plt
from pylab import *

Pylians' Readgadget method, which is included since the module was unavailable for a while

In [ ]:
# routines for reading headers and data blocks from Gadget snapshot files
# usage e.g.:
#
# import readsnap as rs
# header = rs.snapshot_header("snap_063.0") # reads snapshot header
# print header.massarr
# mass = rs.read_block("snap_063","MASS",parttype=5) # reads mass for particles of type 5, using block names should work for both format 1 and 2 snapshots
# print "mass for", mass.size, "particles read"
# print mass[0:10]
#
# before using read_block, make sure that the description (and order if using format 1 snapshot files) of the data blocks
# is correct for your configuration of Gadget
#
# for mutliple file snapshots give e.g. the filename "snap_063" rather than "snap_063.0" to read_block
# for snapshot_header the file number should be included, e.g."snap_063.0", as the headers of the files differ
#
# the returned data block is ordered by particle species even when read from a multiple file snapshot


# ----- class for snapshot header -----

class snapshot_header:
  def __init__(self, filename):

    if os.path.exists(filename):
      curfilename = filename
    elif os.path.exists(filename+".0"):
      curfilename = filename+".0"
    else:
      print("file not found:", filename)
      sys.exit()

    self.filename = filename
    f = open(curfilename,'rb')
    blocksize = np.fromfile(f,dtype=np.int32,count=1)
    if blocksize[0] == 8:
      swap = 0
      format = 2
    elif blocksize[0] == 256:
      swap = 0
      format = 1
    else:
      blocksize.byteswap(True)
      if blocksize[0] == 8:
        swap = 1
        format = 2
      elif blocksize[0] == 256:
        swap = 1
        format = 1
      else:
        print("incorrect file format encountered when reading header of", filename)
        sys.exit()

    self.format = format
    self.swap = swap

    if format==2:
      f.seek(16, os.SEEK_CUR)

    self.npart = np.fromfile(f,dtype=np.int32,count=6)
    self.massarr = np.fromfile(f,dtype=np.float64,count=6)
    self.time = (np.fromfile(f,dtype=np.float64,count=1))[0]
    self.redshift = (np.fromfile(f,dtype=np.float64,count=1))[0]
    self.sfr = (np.fromfile(f,dtype=np.int32,count=1))[0]
    self.feedback = (np.fromfile(f,dtype=np.int32,count=1))[0]
    self.nall = np.fromfile(f,dtype=np.uint32,count=6)
    self.cooling = (np.fromfile(f,dtype=np.int32,count=1))[0]
    self.filenum = (np.fromfile(f,dtype=np.int32,count=1))[0]
    self.boxsize = (np.fromfile(f,dtype=np.float64,count=1))[0]
    self.omega_m = (np.fromfile(f,dtype=np.float64,count=1))[0]
    self.omega_l = (np.fromfile(f,dtype=np.float64,count=1))[0]
    self.hubble = (np.fromfile(f,dtype=np.float64,count=1))[0]

    if swap:
      self.npart.byteswap(True)
      self.massarr.byteswap(True)
      self.time = self.time.byteswap()
      self.redshift = self.redshift.byteswap()
      self.sfr = self.sfr.byteswap()
      self.feedback = self.feedback.byteswap()
      self.nall.byteswap(True)
      self.cooling = self.cooling.byteswap()
      self.filenum = self.filenum.byteswap()
      self.boxsize = self.boxsize.byteswap()
      self.omega_m = self.omega_m.byteswap()
      self.omega_l = self.omega_l.byteswap()
      self.hubble = self.hubble.byteswap()

    f.close()

# ----- find offset and size of data block -----

def find_block(filename, format, swap, block, block_num, only_list_blocks=False):
  if (not os.path.exists(filename)):
      print("file not found:", filename)
      sys.exit()

  f = open(filename,'rb')
  f.seek(0, os.SEEK_END)
  filesize = f.tell()
  f.seek(0, os.SEEK_SET)

  found = False
  curblock_num = 1
  while ((not found) and (f.tell()<filesize)):
    if format==2:
      f.seek(4, os.SEEK_CUR)
      curblock = f.read(4).decode()
      if (block == curblock):
        found = True
      f.seek(8, os.SEEK_CUR)
    else:
      if curblock_num==block_num:
        found = True

    curblocksize = (np.fromfile(f,dtype=np.uint32,count=1))[0]
    if swap:
      curblocksize = curblocksize.byteswap()

    # - print some debug info about found data blocks -
    #if format==2:
    #  print curblock, curblock_num, curblocksize
    #else:
    #  print curblock_num, curblocksize

    if only_list_blocks:
      if format==2:
        print(curblock_num,curblock,f.tell(),curblocksize)
      else:
        print(curblock_num,f.tell(),curblocksize)
      found = False


    if found:
      blocksize = curblocksize
      offset = f.tell()
    else:
      f.seek(curblocksize, os.SEEK_CUR)
      blocksize_check = (np.fromfile(f,dtype=np.uint32,count=1))[0]
      if swap: blocksize_check = blocksize_check.byteswap()
      if (curblocksize != blocksize_check):
        print("something wrong")
        sys.exit()
      curblock_num += 1
  f.close()

  if ((not found) and (not only_list_blocks)):
    print("Error: block not found")
    sys.exit()

  if (not only_list_blocks):
    return offset,blocksize

# ----- read data block -----
#for snapshots with very very large number of particles set nall manually
#for instance nall=np.array([0,2048**3,0,0,0,0])
def read_block(filename, block, parttype=-1, physical_velocities=True,
    arepo=0, no_masses=False, verbose=False, nall=[0,0,0,0,0,0]):

  if (verbose):  print("reading block", block)

  blockadd=0
  blocksub=0

  if arepo==0:
    if (verbose):  print("Gadget format")
    blockadd=0
  if arepo==1:
    if (verbose):  print("Arepo format")
    blockadd=1
  if arepo==2:
    if (verbose):  print("Arepo extended format")
    blockadd=4
  if no_masses==True:
    if (verbose):  print("No mass block present")
    blocksub=1

  if parttype not in [-1,0,1,2,3,4,5]:
    print("wrong parttype given");  sys.exit()

  if os.path.exists(filename):
    curfilename = filename
    single_file = True
  elif os.path.exists(filename+".0"):
    curfilename = filename+".0"
    single_file = False
  else:
    print("file not found:", filename)
    print("and:", curfilename);  sys.exit()

  head = snapshot_header(curfilename)
  format   = head.format
  swap     = head.swap
  npart    = head.npart
  massarr  = head.massarr
  filenum  = head.filenum
  redshift = head.redshift
  time     = head.time
  if np.all(nall==[0,0,0,0,0,0]):
    nall = head.nall
  if verbose:  print("FORMAT=", format)
  del head

  # - description of data blocks -
  # add or change blocks as needed for your Gadget version
  data_for_type = np.zeros(6,bool) # should be set to "True" below for the species for which data is stored in the data block #by doing this, the default value is False data_for_type=[False,False,False,False,False,False]
  dt = np.float32 # data type of the data in the block
  if block=="POS ":
    data_for_type[:] = True
    dt = np.dtype((np.float32,3))
    block_num = 2
  elif block=="VEL ":
    data_for_type[:] = True
    dt = np.dtype((np.float32,3))
    block_num = 3
  elif block=="ID  ":
    data_for_type[:] = True
    dt = np.uint32
    block_num = 4
#only used for format I, when file structure is HEAD,POS,VEL,ID,ACCE
  elif block=="ACCE":              #This is only for the PIETRONI project
    data_for_type[:] = True        #This is only for the PIETRONI project
    dt = np.dtype((np.float32,3))  #This is only for the PIETRONI project
    block_num = 5                  #This is only for the PIETRONI project
  elif block=="MASS":
    data_for_type[np.where(massarr==0)] = True
    block_num = 5
    if parttype>=0 and massarr[parttype]>0:
        if (verbose):    print("filling masses according to massarr")
        if single_file:
          return np.ones(npart[parttype],dtype=dt)*massarr[parttype]
        else:
          return np.ones(nall[parttype],dtype=dt)*massarr[parttype]
  elif block=="U   ":
    data_for_type[0] = True
    block_num = 6-blocksub
  elif block=="RHO ":
    data_for_type[0] = True
    block_num = 7-blocksub
  elif block=="VOL ":
    data_for_type[0] = True
    block_num = 8-blocksub
  elif block=="CMCE":
    data_for_type[0] = True
    dt = np.dtype((np.float32,3))
    block_num = 9-blocksub
  elif block=="AREA":
    data_for_type[0] = True
    block_num = 10-blocksub
  elif block=="NFAC":
    data_for_type[0] = True
    dt = np.dtype(np.int64)        #depends on code version, most recent hast int32, old MyIDType
    block_num = 11-blocksub
  elif block=="NE  ":
    data_for_type[0] = True
    block_num = 8+blockadd-blocksub
  elif block=="NH  ":
    data_for_type[0] = True
    block_num = 9+blockadd-blocksub
  elif block=="HSML":
    data_for_type[0] = True
    block_num = 10+blockadd-blocksub
  elif block=="SFR ":
    data_for_type[0] = True
    block_num = 11+blockadd-blocksub
  elif block=="MHI ":                  #This is only for the bias_HI project
    data_for_type[0] = True            #This is only for the bias_HI project
    block_num = 12+blockadd-blocksub   #This is only for the bias_HI project
  elif block=="TEMP":                  #This is only for the bias_HI project
    data_for_type[0] = True            #This is only for the bias_HI project
    block_num = 13+blockadd-blocksub   #This is only for the bias_HI project
  elif block=="AGE ":
    data_for_type[4] = True
    block_num = 12+blockadd-blocksub
  elif block=="Z   ":
    data_for_type[0] = True
    data_for_type[4] = True
    block_num = 13+blockadd-blocksub
  elif block=="BHMA":
    data_for_type[5] = True
    block_num = 14+blockadd-blocksub
  elif block=="BHMD":
    data_for_type[5] = True
    block_num = 15+blockadd-blocksub
  else:
    print("Sorry! Block type", block, "not known!")
    sys.exit()
  # - end of block description -

  actual_data_for_type = np.copy(data_for_type)
  if parttype >= 0:
    actual_data_for_type[:] = False
    actual_data_for_type[parttype] = True
    if data_for_type[parttype]==False:
      print("Error: no data for specified particle type", parttype, "in the block", block)
      sys.exit()
  elif block=="MASS":
    actual_data_for_type[:] = True

  allpartnum = np.int64(0)
  species_offset = np.zeros(6,np.int64)
  for j in range(6):
    species_offset[j] = allpartnum
    if actual_data_for_type[j]:
        if single_file:
            allpartnum += npart[j]
        else:
          allpartnum += nall[j]

  # define the array containing the information
  data = np.empty(allpartnum,dt)

  # loop over all subfiles
  for i in range(filenum):

    if not(single_file) and filenum>1:
        curfilename = filename+"."+str(i)

    if i>0:
      head  = snapshot_header(curfilename)
      npart = head.npart
      del head

    curpartnum = np.int32(0)
    cur_species_offset = np.zeros(6,np.int64)
    for j in range(6):
      cur_species_offset[j] = curpartnum
      if data_for_type[j]:
        curpartnum += npart[j]

    if parttype>=0:
      actual_curpartnum = npart[parttype]
      add_offset = cur_species_offset[parttype]
    else:
      actual_curpartnum = curpartnum
      add_offset = np.int32(0)

    offset,blocksize = find_block(curfilename,format,swap,block,block_num)

    if i==0: # fix data type for ID if long IDs are used
      if block=="ID  ":
        if blocksize == np.dtype(dt).itemsize*curpartnum * 2:
          dt = np.uint64

    if np.dtype(dt).itemsize*curpartnum != blocksize:
      print("something wrong with blocksize! expected =",np.dtype(dt).itemsize*curpartnum,"actual =",blocksize)
      sys.exit()

    f = open(curfilename,'rb')
    f.seek(offset + add_offset*np.dtype(dt).itemsize, os.SEEK_CUR)
    curdat = np.fromfile(f,dtype=dt,count=actual_curpartnum) # read data
    f.close()
    if swap:  curdat.byteswap(True)

    for j in range(6):
      if actual_data_for_type[j]:
        if block=="MASS" and massarr[j]>0: # add mass block for particles for which the mass is specified in the snapshot header
          data[species_offset[j]:species_offset[j]+npart[j]] = massarr[j]
        else:
          if parttype>=0:
            data[species_offset[j]:species_offset[j]+npart[j]] = curdat
          else:
            data[species_offset[j]:species_offset[j]+npart[j]] = curdat[cur_species_offset[j]:cur_species_offset[j]+npart[j]]
        species_offset[j] += npart[j]

    del curdat

    if single_file:  break


  if physical_velocities and block=="VEL " and redshift!=0:
    data *= math.sqrt(time)

  return data

# ----- list all data blocks in a format 2 snapshot file -----

def list_format2_blocks(filename):
  if   os.path.exists(filename):        curfilename = filename
  elif os.path.exists(filename+".0"):   curfilename = filename+".0"
  else:
    print("file not found:", filename);  sys.exit()

  head   = snapshot_header(curfilename)
  format = head.format
  swap   = head.swap;  del head

  print('GADGET FORMAT ',format)
  if (format != 2):  print("#   OFFSET   SIZE")
  else:              print("#   BLOCK   OFFSET   SIZE")
  print("-------------------------")

  find_block(curfilename, format, swap, "XXXX", 0, only_list_blocks=True)

  print("-------------------------")

def read_gadget_header(filename):
  if   os.path.exists(filename):      curfilename = filename
  elif os.path.exists(filename+".0"): curfilename = filename+".0"
  else:
    print("file not found:", filename);  sys.exit()

  head=snapshot_header(curfilename)
  print('npar=',head.npart)
  print('nall=',head.nall)
  print('a=',head.time)
  print('z=',head.redshift)
  print('masses=',head.massarr*1e10,'Msun/h')
  print('boxsize=',head.boxsize,'kpc/h')
  print('filenum=',head.filenum)
  print('cooling=',head.cooling)
  print('Omega_m,Omega_l=',head.omega_m,head.omega_l)
  print('h=',head.hubble,'\n')

  rhocrit=2.77536627e11 #h**2 M_sun/Mpc**3
  rhocrit=rhocrit/1e9 #h**2M_sun/kpc**3

  Omega_CDM=head.nall[1]*head.massarr[1]*1e10/(head.boxsize**3*rhocrit)
  print('DM mass=%.5e  Omega_DM = %.5f'\
    %(head.massarr[1]*1e10, Omega_CDM))
  if head.nall[2]>0 and head.massarr[2]>0:
    Omega_NU=head.nall[2]*head.massarr[2]*1e10/(head.boxsize**3*rhocrit)
    print('NU mass=%.5e  Omega_NU = %.5f'\
        %(head.massarr[2]*1e10, Omega_NU))
    print('Sum of neutrino masses=%.5f eV'\
        %(Omega_NU*head.hubble**2*94.1745))

In [ ]:
# find snapshot name and format
def fname_format(snapshot):
    if os.path.exists(snapshot):
        if snapshot[-4:]=='hdf5':  filename, fformat = snapshot, 'hdf5'
        else:                      filename, fformat = snapshot, 'binary'
    elif os.path.exists(snapshot+'.0'):
        filename, fformat = snapshot+'.0', 'binary'
    elif os.path.exists(snapshot+'.hdf5'):
        filename, fformat = snapshot+'.hdf5', 'hdf5'
    elif os.path.exists(snapshot+'.0.hdf5'):
        filename, fformat = snapshot+'.0.hdf5', 'hdf5'
    else:  raise Exception('File not found!')
    return filename,fformat


# This class reads the header of the gadget file
class header:
    def __init__(self, snapshot):

        filename, fformat = fname_format(snapshot)

        if fformat=='hdf5':
            f             = h5py.File(filename, 'r')

            self.time     = f['Header'].attrs[u'Time']
            self.redshift = f['Header'].attrs[u'Redshift']
            self.npart    = (f['Header'].attrs[u'NumPart_ThisFile']).astype(np.int64)
            self.nall     = (f['Header'].attrs[u'NumPart_Total']).astype(np.int64)
            self.filenum  = int(f['Header'].attrs[u'NumFilesPerSnapshot'])
            self.massarr  = f['Header'].attrs[u'MassTable']
            self.boxsize  = f['Header'].attrs[u'BoxSize']

            # check if it is a SWIFT snapshot
            if '/Cosmology' in f.keys():
                self.omega_m  = f['Cosmology'].attrs[u'Omega_m']
                self.omega_l  = f['Cosmology'].attrs[u'Omega_lambda']
                self.hubble   = f['Cosmology'].attrs[u'h']

            # check if it is a Gadget-4 snapshot
            elif '/Parameters' in f.keys():
                self.omega_m  = f['Parameters'].attrs[u'Omega0']
                self.omega_l  = f['Parameters'].attrs[u'OmegaLambda']
                self.hubble   = f['Parameters'].attrs[u'HubbleParam']
                #self.cooling  = f['Parameters'].attrs[u'Flag_Cooling']

            # if it is a traditional Gadget-1/2/3 snapshot
            else:
                self.omega_m  = f['Header'].attrs[u'Omega0']
                self.omega_l  = f['Header'].attrs[u'OmegaLambda']
                self.hubble   = f['Header'].attrs[u'HubbleParam']
                #self.cooling  = f['Header'].attrs[u'Flag_Cooling']

            self.format   = 'hdf5'
            f.close()

        else:
            head = snapshot_header(filename)
            self.time     = head.time
            self.redshift = head.redshift
            self.boxsize  = head.boxsize
            self.filenum  = head.filenum
            self.omega_m  = head.omega_m
            self.omega_l  = head.omega_l
            self.hubble   = head.hubble
            self.massarr  = head.massarr
            self.npart    = head.npart
            self.nall     = head.nall
            self.cooling  = head.cooling
            self.format   = head.format

        # km/s/(Mpc/h)
        self.Hubble = 100.0*np.sqrt(self.omega_m*(1.0+self.redshift)**3+self.omega_l)


# This function reads a block of an individual file of a gadget snapshot
def read_field(snapshot, block, ptype):

    filename, fformat = fname_format(snapshot)
    head              = header(filename)
    Masses            = head.massarr*1e10 #Msun/h
    Npart             = head.npart        #number of particles in the subfile
    Nall              = head.nall         #total number of particles in the snapshot

    if fformat=="binary":
        return read_block(filename, block, parttype=ptype)
    else:
        prefix = 'PartType%d/'%ptype
        f = h5py.File(filename, 'r')
        if   block=="POS ":  suffix = "Coordinates"
        elif block=="MASS":  suffix = "Masses"
        elif block=="ID  ":  suffix = "ParticleIDs"
        elif block=="VEL ":  suffix = "Velocities"
        else: raise Exception('block not implemented in readgadget!')

        if '%s%s'%(prefix,suffix) not in f.keys():
            if Masses[ptype] != 0.0:
                array = np.ones(Npart[ptype], np.float32)*Masses[ptype]
            else:
                raise Exception('Problem reading the block %s'%block)
        else:
            array = f[prefix+suffix][:]
        f.close()

        if block=="VEL ":  array *= np.sqrt(head.time)
        if block=="POS " and array.dtype==np.float64:
            array = array.astype(np.float32)

        return array

# This function reads a block from an entire gadget snapshot (all files)
# it can read several particle types at the same time.
# ptype has to be a list. E.g. ptype=[1], ptype=[1,2], ptype=[0,1,2,3,4,5]
def read_gadget_block(snapshot, block, ptype, verbose=False):

    # find the format of the file and read header
    filename, fformat = fname_format(snapshot)
    head    = header(filename)
    Nall    = head.nall
    filenum = head.filenum

    # find the total number of particles to read
    Ntotal = 0
    for i in ptype:
        Ntotal += Nall[i]

    # find the dtype of the block
    if   block=="POS ":  dtype=np.dtype((np.float32,3))
    elif block=="VEL ":  dtype=np.dtype((np.float32,3))
    elif block=="MASS":  dtype=np.float32
    elif block=="ID  ":  dtype=read_field(filename, block, ptype[0]).dtype
    else: raise Exception('block not implemented in readgadget!')

    # define the array containing the data
    array = np.zeros(Ntotal, dtype=dtype)


    # do a loop over the different particle types
    offset = 0
    for pt in ptype:

        # format I or format II Gadget files
        if fformat=="binary":
            array[offset:offset+Nall[pt]] = \
                read_block(snapshot, block, pt, verbose=verbose)
            offset += Nall[pt]

        # single files (either binary or hdf5)
        elif filenum==1:
            array[offset:offset+Nall[pt]] = read_field(snapshot, block, pt)
            offset += Nall[pt]

        # multi-file hdf5 snapshot
        else:

            # do a loop over the different files
            for i in range(filenum):

                # find the name of the file to read
                filename = '%s.%d.hdf5'%(snapshot,i)

                # read number of particles in the file and read the data
                npart = header(filename).npart[pt]
                array[offset:offset+npart] = read_field(filename, block, pt)
                offset += npart

    if offset!=Ntotal:  raise Exception('not all particles read!!!!')

    return array

Readig the snapshot

In [ ]:
snapshot = '/content/drive/MyDrive/Data/snap_004'
ptype    =  [1]
h = header(snapshot)
BoxSize  = h.boxsize/1e3  #Mpc/h
print(BoxSize)
redshift = h.redshift     #redshift of the snapshot
Masses   = h.massarr*1e10 #Masses of the particles in Msun/h
# read positions, velocities and IDs of the particles
pos = read_gadget_block(snapshot, "POS ", ptype)/1e3 #positions in Mpc/h
print(len(pos))

In [ ]:
posarr = np.array(pos)
indexes0 = np.where((pos[:,0]<50) & (pos[:,1]<50) & (pos[:,2]<50))
pos_slide0 = pos[indexes0]
print(len(pos_slide0))

In [ ]:
#3D for xyz
pslides = []
for i in range(11, 21): #Set this to 0, 21 for the full 8000 cubes
  for j in range(21): #20 by 20 for the x-z face -> 20x20x20 for xyz
  #temp_list = [] #Increment x in the outer loop
    for k in range(21): #Increment z in the inner loop
      temp_index = np.where((pos[:,0]<(50*(j+1))) & (pos[:,0]>(50*(j+1))-50)  & (pos[:,1]<50*(i+1)) & (pos[:,1]>(50*(i+1))-50) & (pos[:,2]<(50*(k+1))) & (pos[:,2]>(50*(k+1)-50)))
      #temp_index = np.where((pos[:,0]<50*(i+1)) & (pos[:,2]<50*j) & (pos[:,2]>50*(j-50)))
      pslides.append(pos[temp_index])
      print(i, j, k)
    #pslides.append(temp_list)
  #i = i + 1

In [ ]:
pslides1arr = np.array(pslides1, dtype=object)
pslides1arr[0].shape

In [ ]:
np.save("pslidesarr1", pslides1arr)

2 arrays were created since the method to gather the data took so long. The above cell only has 1.

In [ ]:
arr1 = np.load("pslidesarr1", allow_pickle=True)
#arr2 = np.load("pslides10to20.npy", allow_pickle=True)

In [ ]:
print(arr1.shape,arr2.shape)

(4000,) (4001,)


This cell concatenates the arrays

In [ ]:
final = np.concatenate((arr1,arr2), axis=0)
final.shape

(8001,)

Finding the point clouds between 15000 and 17000 points

In [ ]:
count = 0
for i in range(8001):
    if final[i].shape[0]>=15000 and final[i].shape[0]<=17000:
        count = count+1

count

1494

Storing the indices that are to be deleted

In [ ]:
del_elem = []

for i in range(8001):
    if not(final[i].shape[0]>=15000 and final[i].shape[0]<=17000):
        del_elem.append(i)

In [ ]:
final = np.delete(final,del_elem,axis=0)

In [ ]:
final.shape

(1494,)

In [ ]:
final = np.load('quijote.npy', allow_pickle=True)

Randomly sampling particles to get 15000 per box

In [ ]:
def random_sample_array(arr, sample_size):
    """Randomly sample 'sample_size' elements from each array in 'arr'."""
    num_arrays = arr.shape[0]
    sampled_array = np.zeros((num_arrays, sample_size, 3))

    for i in range(num_arrays):
        element = arr[i]
        n_points = element.shape[0]
        if n_points <= sample_size:
            # If the element has fewer points than the sample_size, use it as is
            sampled_array[i] = element
        else:
            # Randomly sample 'sample_size' points from the element without replacement
            indices = np.random.choice(n_points, sample_size, replace=False)
            sampled_array[i] = element[indices]

    return sampled_array

# Example usage:
if __name__ == "__main__":
    # Example 3D numpy array of shape (551, n, 3)

    # Desired sample size for each element
    desired_sample_size = 15000

    # Randomly sample elements from the array
    quijote = random_sample_array(final, desired_sample_size)

    print("Sampled array shape:", quijote.shape)


In [ ]:
np.save("quijote.npy",quijote)

Loading the dataset for visualisation - the axes need to be swapped otherwise Pylians crashes. This is because it is reading the columns as dimensions, density, not density, dimensions

In [ ]:
dataset = np.load("/content/drive/MyDrive/quijote15k.npy")
dataset = np.swapaxes(dataset, 1, 2)

In [ ]:
np.shape(dataset)

(1494, 3, 15000)

Setting up the plotly graphing objects

In [ ]:
fig = make_subplots(rows=1, cols=5,
                    specs=[[{'type': 'scatter3d'}, {'type': 'scatter3d'}, {'type': 'scatter3d'}, {'type': 'scatter3d'}, {'type': 'scatter3d'}]] )

In [ ]:
for i in range(5):
  fig.add_trace(go.Scatter3d(x=dataset[i][0], y = dataset[i][1], z=dataset[i][2], mode='markers', marker=dict(color='red',size=1)), row=1, col=i+1)
  print(i)


Showing the images from an optimal view

In [ ]:
fig.update_layout(scene1 = dict(xaxis = dict(showgrid = False,showticklabels = False),
                                   yaxis = dict(showgrid = False,showticklabels = False),
                                   zaxis = dict(showgrid = False,showticklabels = False)
             ))
fig.update_layout(scene2 = dict(xaxis = dict(showgrid = False,showticklabels = False),
                                   yaxis = dict(showgrid = False,showticklabels = False),
                                   zaxis = dict(showgrid = False,showticklabels = False)
             ))
fig.update_layout(scene2 = dict(xaxis = dict(showgrid = False,showticklabels = False),
                                   yaxis = dict(showgrid = False,showticklabels = False),
                                   zaxis = dict(showgrid = False,showticklabels = False)
             ))
fig.update_layout(scene3 = dict(xaxis = dict(showgrid = False,showticklabels = False),
                                   yaxis = dict(showgrid = False,showticklabels = False),
                                   zaxis = dict(showgrid = False,showticklabels = False)
             ))
fig.update_layout(scene4 = dict(xaxis = dict(showgrid = False,showticklabels = False),
                                   yaxis = dict(showgrid = False,showticklabels = False),
                                   zaxis = dict(showgrid = False,showticklabels = False)
             ))
fig.update_layout(scene5 = dict(xaxis = dict(showgrid = False,showticklabels = False),
                                   yaxis = dict(showgrid = False,showticklabels = False),
                                   zaxis = dict(showgrid = False,showticklabels = False)
             ))
fig.update_layout(
       height=800,
       width=2000)
fig.layout.scene1.camera.eye=dict(x=2.5, y=2.5, z=2.5)
fig.layout.scene2.camera.eye=dict(x=2.5, y=2.5, z=2.5)
fig.layout.scene3.camera.eye=dict(x=2.5, y=2.5, z=2.5)
fig.layout.scene4.camera.eye=dict(x=2.5, y=2.5, z=2.5)
fig.layout.scene5.camera.eye=dict(x=2.5, y=2.5, z=2.5)
fig.show()

In [ ]:
dataset = np.load("/content/drive/MyDrive/Data/8050epochslinear.npy")

In [ ]:
fig = make_subplots(rows=1, cols=5,
                    specs=[[{'type': 'scatter3d'}, {'type': 'scatter3d'}, {'type': 'scatter3d'}, {'type': 'scatter3d'}, {'type': 'scatter3d'}]] )

In [ ]:
for i in range(5):
  fig.add_trace(go.Scatter3d(x=dataset[i][0], y = dataset[i][1], z=dataset[i][2], mode='markers', marker=dict(color='red',size=1.75)), row=1, col=i+1)
  print(i)


In [ ]:
fig.update_layout(scene1 = dict(xaxis = dict(showgrid = False,showticklabels = False),
                                   yaxis = dict(showgrid = False,showticklabels = False),
                                   zaxis = dict(showgrid = False,showticklabels = False)
             ))
fig.update_layout(scene2 = dict(xaxis = dict(showgrid = False,showticklabels = False),
                                   yaxis = dict(showgrid = False,showticklabels = False),
                                   zaxis = dict(showgrid = False,showticklabels = False)
             ))
fig.update_layout(scene2 = dict(xaxis = dict(showgrid = False,showticklabels = False),
                                   yaxis = dict(showgrid = False,showticklabels = False),
                                   zaxis = dict(showgrid = False,showticklabels = False)
             ))
fig.update_layout(scene3 = dict(xaxis = dict(showgrid = False,showticklabels = False),
                                   yaxis = dict(showgrid = False,showticklabels = False),
                                   zaxis = dict(showgrid = False,showticklabels = False)
             ))
fig.update_layout(scene4 = dict(xaxis = dict(showgrid = False,showticklabels = False),
                                   yaxis = dict(showgrid = False,showticklabels = False),
                                   zaxis = dict(showgrid = False,showticklabels = False)
             ))
fig.update_layout(scene5 = dict(xaxis = dict(showgrid = False,showticklabels = False),
                                   yaxis = dict(showgrid = False,showticklabels = False),
                                   zaxis = dict(showgrid = False,showticklabels = False)
             ))
fig.update_layout(
       height=800,
       width=2000)
fig.layout.scene1.camera.eye=dict(x=2.5, y=2.5, z=2.5)
fig.layout.scene2.camera.eye=dict(x=2.5, y=2.5, z=2.5)
fig.layout.scene3.camera.eye=dict(x=2.5, y=2.5, z=2.5)
fig.layout.scene4.camera.eye=dict(x=2.5, y=2.5, z=2.5)
fig.layout.scene5.camera.eye=dict(x=2.5, y=2.5, z=2.5)
fig.show()

Loading in the result data, which does not need to have the axes swapped

In [ ]:
dataset = np.load("/content/drive/MyDrive/Data/10000epochslinear.npy")
#dataset = np.swapaxes(dataset, 1, 2)

In [ ]:
fig = make_subplots(rows=1, cols=5,
                    specs=[[{'type': 'scatter3d'}, {'type': 'scatter3d'}, {'type': 'scatter3d'}, {'type': 'scatter3d'}, {'type': 'scatter3d'}]] )

In [ ]:
for i in range(5):
  fig.add_trace(go.Scatter3d(x=dataset[i][0], y = dataset[i][1], z=dataset[i][2], mode='markers', marker=dict(color='red',size=1.75)), row=1, col=i+1)
  print(i)


In [ ]:
fig.update_layout(scene1 = dict(xaxis = dict(showgrid = False,showticklabels = False),
                                   yaxis = dict(showgrid = False,showticklabels = False),
                                   zaxis = dict(showgrid = False,showticklabels = False)
             ))
fig.update_layout(scene2 = dict(xaxis = dict(showgrid = False,showticklabels = False),
                                   yaxis = dict(showgrid = False,showticklabels = False),
                                   zaxis = dict(showgrid = False,showticklabels = False)
             ))
fig.update_layout(scene2 = dict(xaxis = dict(showgrid = False,showticklabels = False),
                                   yaxis = dict(showgrid = False,showticklabels = False),
                                   zaxis = dict(showgrid = False,showticklabels = False)
             ))
fig.update_layout(scene3 = dict(xaxis = dict(showgrid = False,showticklabels = False),
                                   yaxis = dict(showgrid = False,showticklabels = False),
                                   zaxis = dict(showgrid = False,showticklabels = False)
             ))
fig.update_layout(scene4 = dict(xaxis = dict(showgrid = False,showticklabels = False),
                                   yaxis = dict(showgrid = False,showticklabels = False),
                                   zaxis = dict(showgrid = False,showticklabels = False)
             ))
fig.update_layout(scene5 = dict(xaxis = dict(showgrid = False,showticklabels = False),
                                   yaxis = dict(showgrid = False,showticklabels = False),
                                   zaxis = dict(showgrid = False,showticklabels = False)
             ))
fig.update_layout(
       height=800,
       width=2000)
fig.layout.scene1.camera.eye=dict(x=2.5, y=2.5, z=2.5)
fig.layout.scene2.camera.eye=dict(x=2.5, y=2.5, z=2.5)
fig.layout.scene3.camera.eye=dict(x=2.5, y=2.5, z=2.5)
fig.layout.scene4.camera.eye=dict(x=2.5, y=2.5, z=2.5)
fig.layout.scene5.camera.eye=dict(x=2.5, y=2.5, z=2.5)
fig.show()